# Bloques Faltantes

Busco el nombre de los bloques faltantes

In [1]:
import difflib

import requests
import pandas as pd
from bs4 import BeautifulSoup

# Bloques Faltantes

Primero, veo si puedo sacar el bloque de los otros años

In [19]:
# Join de todos los otros años
csvs = ['../viajes_2012.csv', '../viajes_2015.csv', '../viajes_2016.csv', '../viajes_2017.csv']

for cnt, csv in enumerate(csvs):
    if cnt == 0:
        df = pd.read_csv(csv)
        fecha = csv.split('_')[1].split('.csv')[0]
        df['Año'] = [fecha for _ in range(df.shape[0])]
    else:
        df_temp = pd.read_csv(csv)
        fecha = csv.split('_')[1].split('.csv')[0]
        df_temp['Año'] = [fecha for _ in range(df_temp.shape[0])]
        
        df = pd.concat([df, df_temp], ignore_index=True)
        
df.head()

,Autoridad,Año,Bloque,Destino,Expediente,Fecha_llegada,Fecha_salida,Motivo,Viaticos_Total_Dolar,Viaticos_Total_Pesos,Viaticos_dolar,Viaticos_euro,Viaticos_pesos
0,"BERMEJO, Rolando Adolfo",2012,PJ Frente para la Victoria,Chile,HSN 0204/2012,08-02-2012,02-02-2012,Feria Internacional de Integración Papudo,1000.000000,4350.0,1000.0,0.0,0.0
1,"BIANCALANI, Fabio Dario",2012,PJ Frente para la Victoria,México,HSN 0284/2012,26-02-2012,19-02-2012,Eurolat,1493.839836,7275.0,1500.0,0.0,0.0
2,"CORRADI, Ana María",2012,PJ Frente para la Victoria,Costa Rica,HSN 0569/2012,08-03-2012,03-03-2012,Parlaméricas,1195.071869,5820.0,1200.0,0.0,0.0
3,"ITURREZ DE CAPPELLINI, Ada Rosa del Valle",2012,Frente Cívico por Santiago,Costa Rica,HSN 0496/2012,11-03-2012,05-03-2012,Parlaméricas,1400.000000,6097.0,1400.0,0.0,0.0
4,"CASTILLO, Oscar Aníbal",2012,Frente Cívico y Social de Catamarca,Francia,HSN 0652/2012,18-03-2012,11-03-2012,World Water Forum,2725.256674,13272.0,0.0,2100.0,0.0


In [29]:
def search_block(ele):
    df_filter = df[df['Autoridad'] == ele]
    amount_blocks = len(set(df_filter['Bloque']))
    
    if amount_blocks == 1:
        return df_filter['Bloque'].iloc[0]
    elif amount_blocks == 0:
        return "No encontrado"
    else:
        return False

block = df_2013['Autoridad'].apply(search_block)

print("Senadores sin bloque en el 2013: {0}. Senadores encontrados: {1}".format(df_2013.shape[0],
                                                                               block[block == "No encontrado"].shape[0]))

Senadores sin bloque en el 2013: 115. Senadores encontrados: 60


# Scrapeo del Sitio de Senadores

Los bloques faltantes los vamos a obtener del sitio oficial del [Senado](http://www.senado.gov.ar). En particular, el sitio tiene [este lugar](http://www.senado.gov.ar/senadores/Historico/PeriodoResultado) que nos permite buscar por el nombre del senador.

Problemas:
* El sitio anda mucho más lento con https (no se por que)
* El server, medio seguido, no te devuelve nada y tenes que hacer otro get (?)


Lo que vamos a hacer, es una pequeña "API" para utilizar esta herramienta de busquede de los senadores

In [12]:
# Header para el request para parecer un usuario "comun"
headers = {
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'User-Agent': ('Mozilla/5.0 (X11; Linux x86_64; rv:45.0)'
                   ' Gecko/20100101 Firefox/45.0'),
}

base_url = "http://www.senado.gov.ar/senadores/Historico/PeriodoResultado"

# Session for the delicious cookies
session = requests.session()
session.headers = headers

# Get the ID of each senador
response = session.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

senadors_id = {}
select = soup.find('select', {'id': 'senado_senadoresbundle_busquedahistoricostype_senador'})
for option in select:
    if option.attrs['value']:
        name =  ' '.join(option.text.replace(',', ' ').split()).lower()
        senadors_id[name] = option.attrs['value']

## "API" Senadores

Es bastante simple. Dado un nombre de Sendanor, buscamos el ID correspondiente. Si no encontramos el nombre del Senador, devolvemos los 3 nombres más cercanos. Si lo encuentra, devuelve un DataFrame de Pandas con los datos de sus periodos

In [28]:
name = ' '.join('MARINO, Juan Carlos'.lower().replace(',', ' ').split())

if name in senadors_id.keys():
    # Senador encontrado
    senador_id = senadors_id[name]
    print(senador_id)
else:
    print(difflib.get_close_matches(name, senadors_id.keys()))

344


In [57]:
def get_block(row):
    name = ' '.join(row['Autoridad'].lower().replace(',', ' ').split())

    if name in senadors_id.keys():
        # Senador encontrado
        senador_id = senadors_id[name]
        r = session.post(base_url, {'senado_senadoresbundle_busquedahistoricostype[senador]': senador_id})
        d = pd.read_html(r.text)[0]

        # Paso las fehcas a datetime
        legal_split = d['Período Legal'].str.split('al', expand=True)
        d['Período Legal Comienzo'] = legal_split[0]
        d['Período Legal Fin'] = legal_split[1]
        d['Período Legal Comienzo'] = pd.to_datetime(d['Período Legal Comienzo'], infer_datetime_format=True)
        d['Período Legal Fin'] = pd.to_datetime(d['Período Legal Fin'], infer_datetime_format=True)

        # Busco la fecha de interes
        date = pd.to_datetime(row['Fecha_salida'], infer_datetime_format=True)
        block = d[(d['Período Legal Comienzo'] < date) & (date < d['Período Legal Fin'])]['Partido Político']
        return block

    else:
        return (difflib.get_close_matches(name, senadors_id.keys()))
    
df_2013[:5].apply(get_block, axis=1)

ValueError: arrays must all be same length

In [54]:
d

,Senador,Nº de Orden,Período Legal,Período Real,Provincia,Partido Político,Reemplazo,Observaciones,Período Legal Comienzo,Período Legal Fin
0,MARINO JUAN CARLOS,344,10-12-2003 al 09-12-2009,10-12-2003 al 09-12-2009,LA PAMPA,UNIÓN CÍVICA RADICAL,MARINO JUAN CARLOS,REEMPLAZA AL SENADOR JUAN CARLOS PASSO,2003-10-12,2009-09-12
1,MARINO JUAN CARLOS,344,10-12-2009 al 09-12-2015,10-12-2009 al 09-12-2015,LA PAMPA,ALIANZA FRENTE PAMPEANO CÍVICO Y SOCIAL,NaN,Sin Observaciones,2009-10-12,2015-09-12
2,MARINO JUAN CARLOS,344,10-12-2015 al 09-12-2021,10-12-2015 al Sin Fecha,LA PAMPA,ALIANZA CAMBIEMOS,MARINO JUAN CARLOS,Sin Observaciones,2015-10-12,2021-09-12
